In [89]:
# !pip install seaborn
# !pip install numpy
# !pip install pandas
# !pip install scikit-learn
# !pip install xgboost

In [91]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler , MaxAbsScaler , MinMaxScaler 
from sklearn.metrics import accuracy_score , classification_report , confusion_matrix
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression , LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
import warnings
import scipy
from sklearn.preprocessing import OrdinalEncoder

In [93]:
df1 = pd.read_csv('auto_train.csv')

In [94]:
df2 = pd.read_csv('20_percent_test_auto.csv')

In [95]:
frames = [df1,df2]
df = pd.concat(frames)

In [96]:
df = df.reset_index(drop=True)

# REMOVING SPACES IN WORDS 

In [20]:
def remove_spaces(x):
    try:
        return x.strip()
    except:
        return np.nan

In [21]:
df["Make"] = df["Make"].apply(remove_spaces)
df["Model"] = df["Model"].apply(remove_spaces)
df["Mileage"] = df["Mileage"].apply(remove_spaces)
df["Body style"] = df["Body style"].apply(remove_spaces)
df["Gearbox"] = df["Gearbox"].apply(remove_spaces)
df["Hand drive"] = df["Hand drive"].apply(remove_spaces)
df["Engine"] = df["Engine"].apply(remove_spaces)
df["Color"] = df["Color"].apply(remove_spaces)
df["Custom cleared"] = df["Custom cleared"].apply(remove_spaces)
df["Price"] = df["Price"].apply(remove_spaces)

# MUVE COLUMNS

In [22]:
def swap_columns(df):
    for i in df[df['Custom cleared'].isna()].index:
                    df.loc[i, 'Custom cleared'] = df.iloc[i]['Color']
                    df.loc[i, 'Color'] = df.iloc[i]['Engine']
                    df.loc[i, 'Engine'] = df.iloc[i]['Hand drive']
                    df.loc[i, 'Hand drive'] = df.iloc[i]['Gearbox']
                    df.loc[i, 'Gearbox'] = df.iloc[i]['Body style']
                    df.loc[i, 'Body style'] = df.iloc[i]['Mileage']
                    df.loc[i, 'Mileage'] = df.iloc[i]['Model']
                    df.loc[i, 'Model'] = np.nan

In [23]:
swap_columns(df)

In [24]:
body_styles = np.array(['Սեդան', 'Մինիվեն / Միկրոավտոբուս', 'Ամենագնաց', 'Պիկապ','Հետչբեք', 'Ունիվերսալ', 'Կուպե', 'Կաբրիոլետ / Ռոդսթեր', 'Ֆուրգոն', 'Մինիվեն', 'Միկրոավտոբուս'])

In [25]:
def swap_body_style_gearbox(df, body_styles):
    for i in df['Body style'].index:
        if df.iloc[i]['Gearbox'] in body_styles:
            df.loc[i, 'Body style'] = df.iloc[i]['Gearbox']
            df.loc[i, 'Gearbox'] = df.iloc[i]['Hand drive']
            df.loc[i, 'Hand drive'] = df.iloc[i]['Engine']
            df.loc[i, 'Engine'] = df.iloc[i]['Color']
            df.loc[i, 'Color'] = np.nan

In [26]:
swap_body_style_gearbox(df, body_styles)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6498 entries, 0 to 6497
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Year            6498 non-null   int64 
 1   Make            6498 non-null   object
 2   Model           6470 non-null   object
 3   Mileage         6498 non-null   object
 4   Body style      6498 non-null   object
 5   Gearbox         6498 non-null   object
 6   Hand drive      6498 non-null   object
 7   Engine          6498 non-null   object
 8   Color           6468 non-null   object
 9   Custom cleared  6498 non-null   object
 10  Price           6498 non-null   object
dtypes: int64(1), object(10)
memory usage: 558.6+ KB


In [28]:
# def swap_body_style_wagon(df):
#     for i in df[df['Body style'] == 'Միկրոավտոբուս'].index:
#         df.loc[i, 'Body style'] = 'Մինիվեն / Միկրոավտոբուս'
# def swap_body_style_minivan(df):
#     for i in df[df['Body style'] == 'Մինիվեն'].index:
#         df.loc[i, 'Body style'] = 'Մինիվեն / Միկրոավտոբուս'

In [29]:
# swap_body_style_wagon(df)
# swap_body_style_minivan(df)

In [30]:
df

,Year,Make,Model,Mileage,Body style,Gearbox,Hand drive,Engine,Color,Custom cleared,Price
0,2018,BMW,745,49001 մղոն,Սեդան,Ավտոմատ,Ձախ,Բենզին,Սպիտակ,մաքսազերծված չ,$ 17 500
1,2019,Tesla,Model 3,17935 կմ,Սեդան,Ավտոմատ,Ձախ,Բենզին,Սպիտակ,մաքսազերծված չ,$ 15 500
2,2000,Opel,Zafira,269000 կմ,Մինիվեն / Միկրոավտոբուս,Մեխանիկական,Ձախ,Գազ,Կապույտ,Մաքսազերծված,֏ 2 220 515
3,2018,Nissan,Maxima,71000 կմ,Սեդան,Ավտոմատ,Ձախ,Բենզին,Սպիտակ,Մաքսազերծված,$ 22 000
4,2020,Volkswagen,Jetta,8227 մղոն,Սեդան,Ավտոմատ,Ձախ,Բենզին,Սպիտակ,մաքսազերծված չ,$ 9 100
...,...,...,...,...,...,...,...,...,...,...,...
6493,2011,VAZ,(Lada) 2107,180000 կմ,Սեդան,Մեխանիկական,Ձախ,Գազ,Արծաթագույն,Մաքսազերծված,֏ 1 750 000
6494,2018,Mazda,3,43009 մղոն,Սեդան,Ավտոմատ,Ձախ,Բենզին,Սև,մաքսազերծված չ,$ 3 700
6495,2017,Kia,Sportage,196852 մղոն,Ամենագնաց,Ավտոմատ,Ձախ,Բենզին,Սև,մաքսազերծված չ,$ 13 000
6496,2017,Toyota,Tundra,23914 կմ,Պիկապ,Ավտոմատ,Ձախ,Բենզին,Սպիտակ,մաքսազերծված չ,$ 13 400


In [31]:
df['Gearbox'] = df['Gearbox'].str.replace('Կիսաավտոմատ', 'Վարիատոր')

# Improving Hand drive

In [32]:
df['Hand drive'] = df['Hand drive'].str.replace('Փոխված աջից ձախ', 'Ձախ')

# Improving Engine

In [33]:
def change_engine(df):
    for i in range(len(df)):
        if df.iloc[i]['Make'] == 'Tesla':
            df.loc[i, 'Engine'] = 'Էլեկտրական'
        

In [34]:
change_engine(df)

# Improving Model

In [35]:
df[df['Model'].isna()]

,Year,Make,Model,Mileage,Body style,Gearbox,Hand drive,Engine,Color,Custom cleared,Price
314,2016,Hyundai,NaN,133000 մղոն,Սեդան,Ավտոմատ,Ձախ,Բենզին,Սպիտակ,մաքսազերծված չ,$ 1 700
815,2018,Hyundai,NaN,128000 մղոն,Սեդան,Ավտոմատ,Ձախ,Բենզին,Մոխրագույն,մաքսազերծված չ,$ 4 000
1010,2019,Hyundai,NaN,59000 մղոն,Սեդան,Ավտոմատ,Ձախ,Բենզին,Սև,մաքսազերծված չ,$ 15 000
1232,2016,Hyundai,NaN,56000 մղոն,Սեդան,Ավտոմատ,Ձախ,Բենզին,Կապույտ,մաքսազերծված չ,$ 2 500
1558,2016,Hyundai,NaN,31000 մղոն,Սեդան,Ավտոմատ,Ձախ,Բենզին,Կարմիր,մաքսազերծված չ,$ 1 900
1595,2017,Hyundai,NaN,103000 մղոն,Սեդան,Ավտոմատ,Ձախ,Բենզին,Սև,մաքսազերծված չ,$ 4 300
1598,2017,Hyundai,NaN,109000 մղոն,Սեդան,Ավտոմատ,Ձախ,Բենզին,Կարմիր,մաքսազերծված չ,$ 2 900
1756,2016,Hyundai,NaN,132000 մղոն,Սեդան,Ավտոմատ,Ձախ,Բենզին,Սև,մաքսազերծված չ,$ 4 000
1872,2017,Hyundai,NaN,84000 մղոն,Սեդան,Ավտոմատ,Ձախ,Հիբրիդ,Կապույտ,մաքսազերծված չ,$ 6 000
2315,2016,Hyundai,NaN,140000 մղոն,Սեդան,Ավտոմատ,Ձախ,Հիբրիդ,Սպիտակ,մաքսազերծված չ,Պայմ.


In [36]:
df.loc[5661 ,'Model']= 'Rio'
df.loc[6002 ,'Model']= 'Sienna'

In [37]:
def change_models_nan(df):
    for i in df[df['Model'].isna()].index:
        if df.loc[i, 'Engine'] == 'Ներքին այրման':
            if df.loc[i, 'Body style'] == 'Սեդան':
                df.loc[i, 'Model'] = 'Elantra'
            else:
                df.loc[i, 'Model'] = 'Accent SE'
        else:
            df.loc[i, 'Model'] = 'Sonata'

In [38]:
change_models_nan(df)

# Improving Mileage

In [39]:
def improve_Milage(x , mile=scipy.constants.mile/1000):
    x = x.strip()
    if "մղոն" in x:
        x = x.strip("մղոն")
        return int(x.strip())*mile
    elif "կմ" in x:
        x = x.strip("կմ")
        return int(x.strip())
    else:
        return np.nan

In [40]:
df["Mileage"] = df["Mileage"].apply(improve_Milage)

In [41]:
df["Mileage"] = df["Mileage"].astype(int)

# Improving Price

In [42]:
def price_changing(df):
    try:
        df['Price'] = df['Price'].str.replace(' ', '')
        dram = []
        euro = []
        for i in df['Price'].index:
            if df.iloc[i]['Price'][0] == '֏':
                dram.append(i)
            elif df.iloc[i]['Price'][0] == '€':
                euro.append(i)
            
        df['Price'] = df['Price'].str.replace(r'[$֏€]', '', regex=True)   
        
        for i in dram:
            df.loc[i,'Price']= int(df.iloc[i]['Price']) * 0.0026
            
        for i in euro:
            df.loc[i,'Price']= int(df.iloc[i]['Price']) * 1.05
    except:
        pass
    


In [43]:
price_changing(df)

In [44]:
df = df.drop(df[df['Price'] == 'Պայմ.'].index)

In [45]:
# try:
#     df["Price"] = df["Price"].astype(int)
# except ValueError:
#         pass

In [46]:
df["Price"] = df["Price"].astype(int)


In [47]:
df = df.drop(df[df['Color'].isna()].index)

In [48]:
df

,Year,Make,Model,Mileage,Body style,Gearbox,Hand drive,Engine,Color,Custom cleared,Price
0,2018,BMW,745,78859,Սեդան,Ավտոմատ,Ձախ,Բենզին,Սպիտակ,մաքսազերծված չ,17500
1,2019,Tesla,Model 3,17935,Սեդան,Ավտոմատ,Ձախ,Էլեկտրական,Սպիտակ,մաքսազերծված չ,15500
2,2000,Opel,Zafira,269000,Մինիվեն / Միկրոավտոբուս,Մեխանիկական,Ձախ,Գազ,Կապույտ,Մաքսազերծված,5773
3,2018,Nissan,Maxima,71000,Սեդան,Ավտոմատ,Ձախ,Բենզին,Սպիտակ,Մաքսազերծված,22000
4,2020,Volkswagen,Jetta,13240,Սեդան,Ավտոմատ,Ձախ,Բենզին,Սպիտակ,մաքսազերծված չ,9100
...,...,...,...,...,...,...,...,...,...,...,...
6493,2011,VAZ,(Lada) 2107,180000,Սեդան,Մեխանիկական,Ձախ,Գազ,Արծաթագույն,Մաքսազերծված,4550
6494,2018,Mazda,3,69216,Սեդան,Ավտոմատ,Ձախ,Բենզին,Սև,մաքսազերծված չ,3700
6495,2017,Kia,Sportage,316802,Ամենագնաց,Ավտոմատ,Ձախ,Բենզին,Սև,մաքսազերծված չ,13000
6496,2017,Toyota,Tundra,23914,Պիկապ,Ավտոմատ,Ձախ,Բենզին,Սպիտակ,մաքսազերծված չ,13400


In [49]:
df = df.reset_index(drop=True)

In [50]:
cleaned_data = df

In [51]:
cleaned_data.to_csv("auto_train_cleaned.csv" , index= False)

In [52]:
df1 = pd.read_csv('auto_train_cleaned.csv')

In [53]:
test = pd.read_csv('auto_data_TEST.csv')

In [54]:
test["Make"] = test["Make"].apply(remove_spaces)
test["Model"] = test["Model"].apply(remove_spaces)
test["Mileage"] = test["Mileage"].apply(remove_spaces)
test["Body style"] = test["Body style"].apply(remove_spaces)
test["Gearbox"] = test["Gearbox"].apply(remove_spaces)
test["Hand drive"] = test["Hand drive"].apply(remove_spaces)
test["Engine"] = test["Engine"].apply(remove_spaces)
test["Color"] = test["Color"].apply(remove_spaces)
test["Custom cleared"] = test["Custom cleared"].apply(remove_spaces)
test["Price"] = test["Price"].apply(remove_spaces)

In [55]:
swap_columns(test)

In [56]:
swap_body_style_gearbox(test, body_styles)

In [57]:
test['Gearbox'] = test['Gearbox'].str.replace('Կիսաավտոմատ', 'Վարիատոր')

In [58]:
test['Hand drive'] = test['Hand drive'].str.replace('Փոխված աջից ձախ', 'Ձախ')

In [59]:
change_engine(test)

In [60]:
change_models_nan(test)

In [61]:
test["Mileage"] = test["Mileage"].apply(improve_Milage)

In [62]:
test["Mileage"] = test["Mileage"].astype(int)

In [63]:
price_changing(test)

In [64]:
test["Price"] = test["Price"].astype(int)


In [65]:
# y_test = test["Price"]

In [66]:
# test["Price"] = np.nan

In [67]:
df = pd.concat([df1 , test])

In [68]:
df = df.reset_index(drop=True)

In [69]:
df

,Year,Make,Model,Mileage,Body style,Gearbox,Hand drive,Engine,Color,Custom cleared,Price
0,2018,BMW,745,78859,Սեդան,Ավտոմատ,Ձախ,Բենզին,Սպիտակ,մաքսազերծված չ,17500
1,2019,Tesla,Model 3,17935,Սեդան,Ավտոմատ,Ձախ,Էլեկտրական,Սպիտակ,մաքսազերծված չ,15500
2,2000,Opel,Zafira,269000,Մինիվեն / Միկրոավտոբուս,Մեխանիկական,Ձախ,Գազ,Կապույտ,Մաքսազերծված,5773
3,2018,Nissan,Maxima,71000,Սեդան,Ավտոմատ,Ձախ,Բենզին,Սպիտակ,Մաքսազերծված,22000
4,2020,Volkswagen,Jetta,13240,Սեդան,Ավտոմատ,Ձախ,Բենզին,Սպիտակ,մաքսազերծված չ,9100
...,...,...,...,...,...,...,...,...,...,...,...
10056,2020,BMW,X5,17327,Ամենագնաց,Ավտոմատ,Ձախ,Բենզին,Արծաթագույն,մաքսազերծված չ,11300
10057,2019,BMW,X3,60000,Ամենագնաց,Ավտոմատ,Ձախ,Բենզին,Սև,մաքսազերծված չ,13000
10058,2016,Mazda,3,21946,Հետչբեք,Ավտոմատ,Ձախ,Բենզին,Սև,մաքսազերծված չ,22300
10059,2016,Toyota,Rav 4,22414,Ամենագնաց,Ավտոմատ,Ձախ,Բենզին,Սպիտակ,մաքսազերծված չ,5600


In [70]:
# df['Price'].fillna(-1, inplace=True)

In [71]:
# df['Price'] = df['Price'].astype(int)

In [72]:
df.drop('Color' , axis = 1 , inplace = True)

In [73]:
encoding_columns = ['Make', 'Model', 'Body style', 'Gearbox', 'Hand drive', 'Engine', 'Custom cleared']

In [74]:
encoder = OrdinalEncoder(dtype=int)

In [75]:
df[encoding_columns] = encoder.fit_transform(df[encoding_columns])

In [76]:
X=df.drop('Price',axis=1)
y=df['Price']

In [77]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error,mean_absolute_error
from sklearn.model_selection import GridSearchCV
X, X_test, y, y_test = train_test_split(X,y, test_size=0.40, random_state=64)

In [78]:
# X = df.iloc[:6319].drop("Price" , axis = 1)
# y = df.iloc[:6319]["Price"]

In [79]:
# X_test = df.iloc[6319:].drop("Price" , axis = 1)
# y_test = df.iloc[6319:]["Price"]

In [82]:
from xgboost import XGBRegressor

In [83]:
model = XGBRegressor()

In [84]:
model.fit(X,y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [85]:
XG_predict=model.predict(X_test)

In [86]:
r2_score(y_test, model.predict(X_test))

0.6711907135194715

In [87]:
MAE = mean_absolute_error(XG_predict,y_test)
MAE

3750.215040404515

In [88]:
1 - MAE / y_test.mean()

0.719844079420052